In [1]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import col, when, unix_timestamp
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("IoT Data Pipeline") \
    .getOrCreate()

# Configuration de la source de données (ici on imagine une source Kafka)




/usr/local/spark/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [ ]:
# Spécifier la configuration de Kafka
kafka_stream_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "iot_data_topic") \
    .load()

# Les données de Kafka sont stockées dans des colonnes 'key' et 'value' sous forme de 'binary'.
# Nous devons décoder les données et les transformer en un DataFrame structuré.

# Définir un schéma pour les données IoT
schema = StructType([
    StructField("machine_id", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("temperature", FloatType(), True),
    StructField("vibration", FloatType(), True),
])

# Décoder les données et les structurer
iot_data_df = kafka_stream_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Affichage des premières lignes
iot_data_df.show(5)


## Nettoyage des données

In [ ]:
# Supprimer les lignes où des valeurs critiques sont manquantes (par exemple, température ou vibration)
iot_data_clean_df = iot_data_df.dropna(subset=["temperature", "vibration"])

# Affichage après nettoyage
iot_data_clean_df.show(5)


## calcul de statut des machines

In [ ]:
# Définir des seuils pour les statuts des machines
iot_data_clean_df = iot_data_clean_df.withColumn(
    "machine_status",
    when((col("temperature") > 40) | (col("vibration") > 1000), "Critique")
    .when((col("temperature") > 35) & (col("temperature") <= 40), "Alerte")
    .otherwise("Normal")
)

iot_data_clean_df.show(5)


## Calcul de la durée de l'état actuel

In [ ]:
# Convertir le timestamp en type datetime pour pouvoir calculer la durée
iot_data_clean_df = iot_data_clean_df.withColumn(
    "timestamp", unix_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss").cast("timestamp")
)

# Calculer la durée en minutes entre chaque événement (fenêtre temporelle)
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy("machine_id").orderBy("timestamp")

iot_data_clean_df = iot_data_clean_df.withColumn(
    "duration_in_current_status",
    (F.unix_timestamp("timestamp") - F.unix_timestamp(F.lag("timestamp", 1).over(windowSpec))) / 60
)

iot_data_clean_df.show(5)


## Détection des anomalies et ajout de métriques supplémentaires

In [ ]:
# Anomalie basée sur une température excessive
iot_data_clean_df = iot_data_clean_df.withColumn(
    "is_anomaly",
    when((col("temperature") > 40) | (col("vibration") > 1000), True).otherwise(False)
)

iot_data_clean_df.show(5)


## Envoi des données transformées vers Elasticsearch

In [ ]:
# Configuration de l'index Elasticsearch
es_config = {
    "es.nodes": "localhost",
    "es.port": "9200",
    "es.index.auto.create": "true",
    "es.write.operation": "index"
}

# Envoi des données transformées vers Elasticsearch en streaming
iot_data_clean_df.writeStream \
    .format("org.elasticsearch.spark.sql") \
    .option("checkpointLocation", "/tmp/checkpoint/") \
    .options(**es_config) \
    .start() \
    .awaitTermination()


In [7]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.elasticsearch:elasticsearch-spark-30_2.12:8.11.0 pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, from_json, unix_timestamp, lag, expr
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType
from pyspark.sql import functions as F

# 1. Initialiser Spark Session
spark = SparkSession.builder \
    .appName("IoTDataProcessing") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# 2. Définir le schéma des données JSON venant de Kafka
schema = StructType() \
    .add("timestamp", StringType()) \
    .add("machine_id", StringType()) \
    .add("region", StringType()) \
    .add("season", StringType()) \
    .add("temperature", DoubleType()) \
    .add("humidity", DoubleType()) \
    .add("soil_moisture", DoubleType()) \
    .add("vibration", DoubleType()) \
    .add("pressure", DoubleType())

# 3. Lire les données Kafka
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:9092") \
    .option("subscribe", "iot_raw_data") \
    .load()

json_df = raw_df.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), schema).alias("data")) \
    .select("data.*")


# Assurez-vous que timestamp est bien en TimestampType
iot_df = json_df.withColumn("timestamp", col("timestamp").cast(TimestampType()))

# Crée une version numérique du timestamp
iot_df = iot_df.withColumn("timestamp_sec", unix_timestamp("timestamp"))

# Ajout des colonnes d’alerte
iot_df = iot_df.withColumn(
    "alerte_temperature", 
    when(col("temperature") > 50, "Alerte").otherwise("Normal")
)

iot_df = iot_df.withColumn(
    "alerte_sol_sec",
    when((col("humidity") < 10) & (col("temperature") > 40), "Critique").otherwise("Normal")
)

iot_df = iot_df.withColumn(
    "alerte_vibration_pression",
    when((col("vibration") > 5) & (col("pressure") > 1.5), "Alerte").otherwise("Normal")
)

# ⚠️ Utiliser une fenêtre basée sur timestamp_sec (BIGINT), pas sur timestamp (TIMESTAMP)
windowSpec = Window \
    .partitionBy("machine_id") \
    .orderBy("timestamp_sec") \
    .rangeBetween(-600, 0)  # 10 minutes en secondes

# Calcul de délai de lecture : max timestamp_sec dans la fenêtre
iot_df = iot_df.withColumn(
    "delai_lecture", 
    col("timestamp_sec") - F.max("timestamp_sec").over(windowSpec)
)

iot_df = iot_df.withColumn(
    "statut_capteur",
    when(col("delai_lecture") > 3600, "Défaillant").otherwise("Actif")
)

# Alerte régionale
alerte_regionale_df = iot_df \
    .filter((col("alerte_temperature") == "Alerte") | (col("alerte_vibration_pression") == "Alerte")) \
    .groupBy("region") \
    .count() \
    .filter(col("count") >= 2) \
    .withColumn("alerte_regionale", F.lit("Alerte régionale")) \
    .select("region", "alerte_regionale")

iot_df = iot_df.join(alerte_regionale_df, on="region", how="left") \
    .fillna({"alerte_regionale": "Rien à signaler"})

# Statut global
iot_df = iot_df.withColumn(
    "statut_global",
    when((col("alerte_sol_sec") == "Critique") | (col("statut_capteur") == "Défaillant"), "Critique")
    .when((col("alerte_temperature") == "Alerte") |
          (col("alerte_vibration_pression") == "Alerte") |
          (col("alerte_regionale") == "Alerte régionale"), "Alerte")
    .otherwise("Normal")
)

# Écriture vers Elasticsearch
iot_df.writeStream \
    .format("org.elasticsearch.spark.sql") \
    .option("checkpointLocation", "/tmp/checkpoint_iot") \
    .option("es.nodes", "es01") \
    .option("es.port", "9200") \
    .option("es.nodes.wan.only", "true") \
    .option("es.net.ssl", "true") \
    .option("es.net.ssl.cert.allow.self.signed", "true") \
    .option("es.net.http.auth.user", "elastic") \
    .option("es.net.http.auth.pass", "Eselpil2") \
    .option("es.resource", "iot_data_enriched/doc") \
    .option("es.mapping.id", "unique_id") \
    .outputMode("update") \
    .start() \
    .awaitTermination()


AnalysisException: [NON_TIME_WINDOW_NOT_SUPPORTED_IN_STREAMING] Window function is not supported in MAX(TIMESTAMP_SEC#875L) (as column `_we0`) on streaming DataFrames/Datasets. Structured Streaming only supports time-window aggregation using the WINDOW function. (window specification: (PARTITION BY MACHINE_ID ORDER BY TIMESTAMP_SEC ASC NULLS FIRST RANGE BETWEEN -600L FOLLOWING AND CURRENT ROW))

In [9]:
from pyspark.sql.types import StructType, StringType, DoubleType
from pyspark.sql.functions import from_json, col

# 1. Définir le schéma
schema = StructType() \
    .add("timestamp", StringType()) \
    .add("machine_id", StringType()) \
    .add("region", StringType()) \
    .add("season", StringType()) \
    .add("temperature", DoubleType()) \
    .add("humidity", DoubleType()) \
    .add("soil_moisture", DoubleType()) \
    .add("vibration", DoubleType()) \
    .add("pressure", DoubleType())

# 2. Lire depuis Kafka
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "host.docker.internal:9092") \
    .option("subscribe", "iot_raw_data") \
    .option("startingOffsets", "latest") \
    .load()

# 3. Convertir la colonne `value` (bytes) en string, puis JSON
json_df = raw_df.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*")

# 4. Écrire dans Elasticsearch (index iot_simplifie)
query = json_df.writeStream \
    .format("org.elasticsearch.spark.sql") \
    .option("checkpointLocation", "/tmp/checkpoint_iot_simple") \
    .option("es.nodes", "es01") \
    .option("es.port", "9200") \
    .option("es.nodes.wan.only", "true") \
    .option("es.net.ssl", "true") \
    .option("es.net.ssl.cert.allow.self.signed", "true") \
    .option("es.net.http.auth.user", "elastic") \
    .option("es.net.http.auth.pass", "Eselpil2") \
    .option("es.resource", "iot_simplifie/_doc") \
    .option("es.nodes.wan.only", "true") \
    .start()

query.awaitTermination()


StreamingQueryException: [STREAM_FAILED] Query [id = b1ee4349-6340-40ee-8496-a7b155b5d48c, runId = 645f9f1a-6539-449e-ad88-101775df5776] terminated with exception: Failed to create new KafkaAdminClient